In [1271]:
# import np plt
import numpy as np
import matplotlib.pyplot as plt

In [1272]:
# Get 10 random vector in 3 dim
X = np.random.randn(10, 3)
X

array([[ 0.12932701, -0.37689875,  0.20321524],
       [-1.09751303,  1.16580213, -1.42514143],
       [ 0.82771336, -0.22597409,  0.05971845],
       [-0.63752622, -0.41239791,  0.19045441],
       [-0.57517192, -0.39100485, -1.73169072],
       [ 1.35332829,  1.11718147, -0.18972555],
       [-0.09174353, -0.34372041, -0.39210899],
       [ 0.04513773,  0.3569891 ,  0.02103514],
       [ 1.34402646,  0.27836556, -1.27175826],
       [ 0.14381797, -0.14802135,  0.76350792]])

In [1273]:
# generate X in such a way that no distance is less than 1
def generate_X():
    while True:
        X = np.random.randn(10, 3)
        if np.min(np.linalg.norm(X, axis=1)) > 1:
            return X

In [1274]:
X = generate_X()
a = 0.01

In [1275]:
# def a func that get dX, so find the distances among the vectors ecc ecc (is like D)
def get_dX(X):
    dX = np.zeros((X.shape[0], X.shape[0], X.shape[1]))
    for i in range(10):
        for j in range(i+1, 10):
            dX[i,j,:] = X[i,:] - X[j,:]
    
    dX = dX - dX.transpose(1,0,2)
    return dX

def get_dx(X):
    dx = np.zeros((X.shape[0], X.shape[0]))
    for i in range(10):
        for j in range(i+1, 10):
            dx[i,j] = np.linalg.norm(X[i,:] - X[j,:])
    
    dx = dx + dx.transpose()
    return dx

# u_prime[i,j] = a/(dx[i,j]**2 - a*dx[i,j])
def get_u_prime(dx, a):
    # exploiting the fact that dx is symmetric
    u_prime = np.zeros_like(dx)
    for i in range(10):
        for j in range(i+1, 10):
            u_prime[i,j] = a/(dx[i,j]**2 - a*dx[i,j])
    
    u_prime = u_prime + u_prime.transpose()
    return u_prime

In [1276]:
X[1,:] - X[2,:]

array([-1.24840338, -1.12924137, -2.0495497 ])

In [1277]:
dX = get_dX(X)
dx = get_dx(X)

In [1278]:
dX[1,2,:]

array([-1.24840338, -1.12924137, -2.0495497 ])

In [1279]:
dX[2,1,:]   

array([1.24840338, 1.12924137, 2.0495497 ])

In [1280]:
# get the norm and compare with dx
np.linalg.norm(dX[1,2,:])


2.6522350980677802

In [1281]:
dx[1,2]

2.6522350980677802

In [1282]:
dx[2,1]

2.6522350980677802

In [1283]:
u_prime = get_u_prime(dx, a)

In [1284]:
u_prime[0,0]

0.0

## Second term

In [1285]:
# grad_phi = -2*alpha * (X[0], X[1], beta*X[2])
def get_grad_phi_over_phi(X, alpha, beta):
    grad_phi = np.zeros_like(X)
    grad_phi[:,0:2] = X[:,0:2]
    grad_phi[:,2] = beta*X[:,2]
    grad_phi = -2*alpha*grad_phi
    
    return grad_phi

In [1286]:
X

array([[ 1.68015566,  0.30602063, -0.32220488],
       [-1.16048316, -0.02723632, -0.66058664],
       [ 0.08792022,  1.10200505,  1.38896306],
       [-1.57670306, -1.28814256, -0.2837489 ],
       [-0.32554206,  0.10721551, -0.96879181],
       [ 2.41268797,  0.29977955, -0.67220324],
       [ 1.6046917 , -0.04342   , -0.01221028],
       [-0.68646014, -0.82569621, -0.89201039],
       [-0.35520507,  1.40859436,  1.23261948],
       [-0.30760291, -1.71353097, -0.04542484]])

In [1287]:
grad_phi_over_phi = get_grad_phi_over_phi(X, 2, 3)
grad_phi_over_phi

array([[ -6.72062263,  -1.22408253,   3.86645859],
       [  4.64193265,   0.10894528,   7.92703966],
       [ -0.35168088,  -4.40802018, -16.66755669],
       [  6.30681225,   5.15257024,   3.40498682],
       [  1.30216825,  -0.42886203,  11.62550177],
       [ -9.6507519 ,  -1.19911818,   8.06643884],
       [ -6.41876681,   0.17367999,   0.14652342],
       [  2.74584058,   3.30278483,  10.70412464],
       [  1.42082029,  -5.63437745, -14.79143374],
       [  1.23041165,   6.85412387,   0.54509809]])

In [1288]:
def get_sec_term(X, dX, dx, u_prime, grad_phi_over_phi):
    sec_term = 0 
    for k in range(X.shape[0]):
        corr_term = np.zeros_like(X[k,:])
        for j in range(X.shape[0]):
            if j != k:
                corr_term += dX[k,j]*u_prime[k,j]/dx[k,j]
        sec_term += 2 * np.dot(grad_phi_over_phi[k,:], corr_term)
    return sec_term

In [1289]:
# get the second term
sec_term = get_sec_term(X, dX, dx, u_prime, grad_phi_over_phi)
sec_term

-2.8269838853111593

In [1290]:
def get_third_term(X, dX, dx, u_prime):
    third_term = 0
    for k in range(X.shape[0]):
        for j in range(X.shape[0]):
            for i in range(X.shape[0]):
                if i != k and j != k:
                    third_term += np.dot(dX[k,j], dX[k,i]) * u_prime[k,j] * u_prime[k,i] / (dx[k,j]*dx[k,i])
    return third_term

In [1291]:
# get the third term
third_term = get_third_term(X, dX, dx, u_prime)
third_term

0.009543199285417536

In [1292]:
def u_second(a, dx):
    return a*(a-2*dx)/(dx**2 - a*dx)**2

In [1293]:
def get_fourth_term(u_prime, dx, a, u_second):
    fourth_term = 0
    for k in range(10):
        for j in range(10):
            if j != k:
                fourth_term += u_second(a, dx[k,j]) + 2 * u_prime[k,j]/ dx[k,j]
    return fourth_term

In [1294]:
# get the fourth term
fourth_term = get_fourth_term(u_prime, dx, a, u_second)
fourth_term


-0.008063163643763901

In [1295]:
# sum all the terms

sum = sec_term + third_term + fourth_term
sum

-2.8255038496695057